In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
from scipy.optimize import curve_fit

## Data preparation

### Load data

In [2]:
chirp_direction = 'chirp_up'
data_file_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_' + f'{chirp_direction}.pkl'
data_cols = [
		'chirp_rate', 'fraction',
		'CA+_mean', 'CA+_std_dev', 'CA+_percentile_0', 'CA+_percentile_10', 'CA+_percentile_20', 
		'CA+_percentile_30', 'CA+_percentile_40', 'CA+_percentile_50', 'CA+_percentile_60', 'CA+_percentile_70', 
		'CA+_percentile_80', 'CA+_percentile_90', 'CA+_percentile_100', 'CA-_mean', 'CA-_std_dev', 'CA-_percentile_0', 
		'CA-_percentile_10', 'CA-_percentile_20', 'CA-_percentile_30', 'CA-_percentile_40', 'CA-_percentile_50', 
		'CA-_percentile_60', 'CA-_percentile_70', 'CA-_percentile_80', 'CA-_percentile_90', 'CA-_percentile_100'
	]
data = pd.DataFrame(pd.read_pickle(data_file_path), columns=data_cols)

# 80-20 train/test split
train_data_df = data.sample(frac=0.8, random_state=0)
test_data_df = data.drop(train_data_df.index)

print(train_data_df.shape)
# print(train_data.head())
print(test_data_df.shape)
# print(test_data.head())

(253, 28)
(63, 28)


## Constant parameters

In [3]:
# I'm not sure of the calculations here, copied from the original code
# constants for evaluation: keff, bigT
speed_of_light_in_vacuum = 299792458
f1 = 384.2304844685e12 - 72.9113e6 - 2.56300597908911e9 + 61.625e6
f2 = f1 + 6.725e9 + 112.06936e6
k1 = 2*np.pi/(speed_of_light_in_vacuum/f1)
k2 = 2*np.pi/(speed_of_light_in_vacuum/f2)
Keff = k1+k2

bigT = 10e-3

## Custom Loss Function

In [4]:
torch.set_default_dtype(torch.float64)

In [5]:
def sine(alpha, contrast, acc_due_to_gravity, fringe_offset):
    #
    phi = (Keff * acc_due_to_gravity - 2 * np.pi * alpha) * (bigT**2)
    return (-contrast * np.cos(phi) + fringe_offset)

In [6]:
def get_popt_for_best_fit(corrected_chirp_rates: np.ndarray, observed_fractions: np.ndarray):
	contrast = (np.max(observed_fractions) - np.min(observed_fractions)) / 2
	fringe_offset = np.mean(observed_fractions)
	g0 = -9.78306 if chirp_direction == 'chirp_down' else 9.77997
	period = 780e-9/(2*bigT**2)

	sine_function_parameters = [contrast, g0, fringe_offset]
	bounds = ([0, g0 - 0.501*period, -np.inf], [1, g0 + 0.501*period, np.inf])
	popt, _ = curve_fit(sine, corrected_chirp_rates, observed_fractions, p0=sine_function_parameters, maxfev=1000000, bounds=bounds)
	return popt

In [7]:
def calculate_mse_loss_with_popt(outputs: torch.TensorType, observed_fractions: torch.TensorType, *popt):
	# print(f'value in t0: {Keff * popt[1] * bigT**2}')
	tensor0 = torch.tensor([Keff * popt[1] * bigT**2 for _ in range(len(outputs))], dtype=torch.float64)
	# print('tensor0')
	# print(tensor0)
	tensor1 = outputs * bigT**2 * 2 * torch.pi
	# print('outputs')
	# print(outputs)
	# print('tensor1')
	# print(tensor1)
	diff_tensor = tensor0 - tensor1
	# print('diff_tensor')
	# print(diff_tensor)
	phi_tensor = torch.cos(diff_tensor)
	# print('phi_tensor')
	# print(phi_tensor)
	predicted_fractions_tensor = phi_tensor * (-popt[0]) + popt[2]
	# print('predicted_fractions_tensor')
	# print(predicted_fractions_tensor)
	mse_loss = nn.MSELoss()
	# print('observed_fractions')
	# print(observed_fractions)
	loss = mse_loss(predicted_fractions_tensor, observed_fractions)
	# print(f'loss: {loss}')
	log_loss = torch.log(loss) * (-1)
	# print(f'log_loss: {log_loss}')
	return log_loss

## Benchmark MSE score

In [8]:
test_chirp_rates = test_data_df['chirp_rate'].to_numpy()
test_fractions = test_data_df['fraction'].to_numpy()
popt = get_popt_for_best_fit(test_chirp_rates, test_fractions)

residuals_baseline = test_fractions - sine(test_chirp_rates, *popt)
mse_baseline = np.mean(residuals_baseline**2)
log_mse_baseline = -np.log(mse_baseline)
print(f'log MSE: {log_mse_baseline}')

log MSE: 4.879983776715633


## Training

### Neural Network

In [9]:
class TransformationNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationNN, self).__init__()
        # Define the layers of the network: 2 fully connected layers 27 -> 64 -> 64 -> 1
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
training_data_tensor = torch.tensor(train_data_df.drop(columns=['fraction']).values)
observed_fractions_tensor = torch.tensor(train_data_df['fraction'].values)
observed_fractions_ndarray = train_data_df['fraction'].to_numpy()

In [11]:
# Initialize the neural network
model = TransformationNN(input_dim=27, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(training_data_tensor)
    outputs_clone = outputs.clone()
    corrected_chirp_rates_ndarray = outputs_clone.detach().cpu().squeeze().numpy()
    
    popt = get_popt_for_best_fit(corrected_chirp_rates_ndarray, observed_fractions_ndarray)    
    loss = calculate_mse_loss_with_popt(outputs.squeeze(), observed_fractions_tensor, *popt)
    if epoch % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], log_MSE: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

Epoch [1/1000], log_MSE: 4.98102941475709
Epoch [101/1000], log_MSE: 4.981030272578259
Epoch [201/1000], log_MSE: 4.981028894874161
Epoch [301/1000], log_MSE: 4.981028210105027
Epoch [401/1000], log_MSE: 4.981054536400692
Epoch [501/1000], log_MSE: 4.981028985991952
Epoch [601/1000], log_MSE: 4.981030260502371
Epoch [701/1000], log_MSE: 4.9810280921014645
Epoch [801/1000], log_MSE: 4.981028782119652
Epoch [901/1000], log_MSE: 4.981030083045838


In [12]:
observed_chirp_rates_for_test_data = test_data_df.drop(columns=['fraction']).values
observed_chirp_rates_for_test_data_tensor = torch.tensor(observed_chirp_rates_for_test_data)
outputs = model(observed_chirp_rates_for_test_data_tensor)
corrected_chirp_rates_ndarray = outputs.detach().cpu().squeeze().numpy()

popt = get_popt_for_best_fit(corrected_chirp_rates_ndarray, test_fractions)
residuals_nn = test_fractions - sine(corrected_chirp_rates_ndarray, *popt)
mse_nn = np.mean(residuals_nn**2)
log_mse_nn = -np.log(mse_nn)
print(f'log_MSE: {log_mse_nn}')
print(f'Improvement: {log_mse_baseline - log_mse_nn}')

log_MSE: 4.880016997549585
Improvement: -3.322083395129738e-05


### Genetic Algorithm